# import

In [1]:
from airbnb_mysql import *
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Connexion to database

In [2]:
port_name = '3306'
user_name = 'root'
user_password = 'root'
host_name = 'localhost'
db = 'airbnb'

connection = create_db_connection(user=user_name, password=user_name, host=host_name, db_name=db, port=port_name)

query = """
        SELECT * FROM apartment;
"""

col_name = ["id", "neighbour", "lat", "long", "r_type", "price", "nbr_rev", "rev_per_month"]
data = db_to_df(connection, query, col_name)

MySQL Database connection successful


# Preprocessing

In [44]:
df = data.drop(["id","nbr_rev","rev_per_month"],axis = 1)

In [40]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import RobustScaler

In [71]:
def encoding(df_col):
    X = df_col
    encoder = LabelBinarizer()
    X_t = encoder.fit_transform(X)
    column = encoder.classes_
    return X_t, column

In [73]:
def scaling(df_col):
    X = df_col
    X = X.to_numpy()
    X = np.reshape(X,(-1,1))
    scaler = RobustScaler().fit(X)
    X_t = scaler.transform(X)
    return X_t

In [74]:
encod_room, room_col = encoding(df['r_type'])
room = pd.DataFrame(encod_room,columns=room_col)


encod_neigh, neigh_col = encoding(df['neighbour'])
neigh = pd.DataFrame(encod_neigh,columns=neigh_col)
scaled_price = scaling(df['price'])

In [79]:
frames = [df,room,neigh]
clean_df = pd.concat(frames,axis=1)
clean_df = clean_df.drop(["neighbour","r_type"],axis=1)

clean_df['scaled_price']= scaled_price
clean_df = clean_df.drop(['price'],axis=1)

clean_df = clean_df.drop(['lat','long'],axis =1)

# Train test split

In [33]:
from sklearn.model_selection import train_test_split

In [80]:
train_set,test_set = train_test_split(clean_df,test_size=0.3,random_state=0)

In [85]:
train_set

pandas.core.frame.DataFrame

# Regression

In [82]:
from sklearn.linear_model import SGDRegressor

In [86]:
y_train = train_set['scaled_price']
X_train = train_set.drop('scaled_price',axis=1)
reg = SGDRegressor(max_iter=1000, tol=1e-3)
reg.fit(X_train,y_train)

SGDRegressor()

In [ ]:
X_test = test_set.drop()